In [ ]:
# --- 1. Library Imports ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Optional: For visualization
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 2. Global Hyperparameters ---
# Model Dimensions (Keep these small for the sprint)
D_MODEL = 64        # Embedding dimension
NUM_HEADS = 4       # Number of attention heads
NUM_LAYERS = 1      # Use 1 Encoder and 1 Decoder layer for simplicity
D_FF = 128          # Feed-Forward hidden dimension

# Training Parameters
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50 

# Sequence Parameters
MAX_SEQ_LEN = 10    # Max tokens in any sequence

In [ ]:
# --- 1. Vocabulary Definition and Mapping ---
# The fixed, small vocabulary for our lookup table
VOCAB = {
    '[PAD]': 0, 
    '[SOS]': 1, 
    '[EOS]': 2,
    
    # Keys
    'COLOR': 3, 'NAME': 4, 'ITEM': 5, 
    
    # Values
    'RED': 6, 'BLUE': 7, 'GREEN': 8,
    'ALICE': 9, 'BOB': 10, 'CHAIR': 11,
    
    # Query Tokens
    'QUERY': 12, 'FIND': 13
}
VOCAB_SIZE = len(VOCAB)
ID_TO_TOKEN = {v: k for k, v in VOCAB.items()}

# --- 2. The Lookup Dictionary (Ground Truth) ---
# Defines all the possible fact pairings the model must learn.
FACTS_DICT = {
    # Key Token ID: Value Token ID
    VOCAB['RED']: VOCAB['BLUE'],    # Red item's value is 'BLUE'
    VOCAB['ALICE']: VOCAB['GREEN'], # Alice's value is 'GREEN'
    VOCAB['CHAIR']: VOCAB['RED']    # Chair's value is 'RED'
}
FACT_PAIRS = list(FACTS_DICT.items()) # List of (Key_ID, Value_ID)

# --- 3. Synthetic Data Generation Function (FIXED) ---
def generate_synthetic_data(num_samples):
    data = []
    
    for _ in range(num_samples):
        # 1. Randomly select facts for the Encoder input (The Dictionary)
        # Select 2 distinct facts to ensure context is required.
        fact1_key, fact1_val = FACT_PAIRS[np.random.randint(len(FACT_PAIRS))]
        
        # 2. Randomly select one fact to be the target (The Query)
        query_key_id = fact1_key
        target_val_id = fact1_val
        
        # Construct Encoder Input (X): The facts (order is randomized later to test no-PE logic)
        enc_input_tokens = [VOCAB['COLOR'], query_key_id, VOCAB['ITEM'], fact1_val] 
        np.random.shuffle(enc_input_tokens) # Randomize order: crucial for no-PE test!
        
        # Add a final token to indicate end of fact list
        enc_input_tokens = [VOCAB['NAME']] + enc_input_tokens 
        
        # Construct Decoder Target (Y): The value we want
        # The target sequence is [Value, EOS]
        # --- FIX APPLIED HERE: Used '[EOS]' instead of 'EOS' ---
        dec_target_tokens = [target_val_id, VOCAB['[EOS]']]
        
        data.append({
            'enc_input': torch.tensor(enc_input_tokens, dtype=torch.long),
            'dec_target': torch.tensor(dec_target_tokens, dtype=torch.long),
            'query_val_id': target_val_id # Used for checking/visualization
        })

    return data

# --- 4. PyTorch Dataset and DataLoader Setup (CORRECTED) ---
class LookupDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        # We need to return encoder input, decoder input (starts with SOS), and true target
        item = self.data[idx]
        
        # Decoder input is the target sequence shifted right (starts with [SOS])
        # FIX: Accessing the token as '[SOS]' instead of 'SOS'
        dec_input = torch.cat([torch.tensor([VOCAB['[SOS]']]), item['dec_target'][:-1]])
        
        # The true target is the original target, which excludes the [SOS] token
        true_target = item['dec_target']
        
        return item['enc_input'], dec_input, true_target

# Collate function to handle padding for the DataLoader
def collate_fn(batch):
    # Padding sequences to MAX_SEQ_LEN
    enc_inputs = [item[0] for item in batch]
    dec_inputs = [item[1] for item in batch]
    dec_targets = [item[2] for item in batch]
    
    # Use torch.nn.utils.rnn.pad_sequence for consistent padding
    enc_inputs = nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True, padding_value=VOCAB['[PAD]'])
    dec_inputs = nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True, padding_value=VOCAB['[PAD]'])
    dec_targets = nn.utils.rnn.pad_sequence(dec_targets, batch_first=True, padding_value=VOCAB['[PAD]'])
    
    # Create padding masks (True means mask, False means keep)
    src_padding_mask = (enc_inputs == VOCAB['[PAD]'])
    tgt_padding_mask = (dec_inputs == VOCAB['[PAD]'])
    
    return enc_inputs, dec_inputs, dec_targets, src_padding_mask, tgt_padding_mask

# Generate and split data
full_data = generate_synthetic_data(num_samples=5000)
train_split = int(0.8 * 5000)
val_split = int(0.1 * 5000)

train_data = full_data[:train_split]
val_data = full_data[train_split:train_split + val_split]
test_data = full_data[train_split + val_split:]

# Create DataLoaders
train_loader = DataLoader(LookupDataset(train_data), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(LookupDataset(val_data), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(LookupDataset(test_data), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
print(f"Data ready: Train={len(train_data)}, Val={len(val_data)}, Test={len(test_data)}")

In [ ]:
# --- 1. Scaled Dot-Product Attention Function (Helper) ---
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Calculates the scaled dot-product attention.
    Q, K, V: [batch_size, n_heads, seq_len, d_k]
    """
    d_k = Q.size(-1)
    # [B, H, Lq, Lk]
    scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

    if mask is not None:
        # Mask is applied before softmax. Masked positions are set to a very large negative number.
        scores = scores.masked_fill(mask == 0, -1e9)

    # [B, H, Lq, Lk]
    attention_weights = torch.softmax(scores, dim=-1)
    
    # [B, H, Lq, Lk] @ [B, H, Lk, Dv] -> [B, H, Lq, Dv]
    output = torch.matmul(attention_weights, V)
    
    return output, attention_weights

# --- 2. Multi-Head Attention (MHA) Class ---
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads # Dimension of K, Q, V for a single head
        
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Linear layers for Q, K, V and the final output
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # 1. Linear projections
        # [B, L, D] -> [B, L, D]
        q = self.W_q(Q)
        k = self.W_k(K)
        v = self.W_v(V)
        
        # 2. Split into heads
        # [B, L, D] -> [B, L, H, d_k] -> [B, H, L, d_k]
        q = q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        
        # 3. Calculate attention
        # x: [B, H, Lq, d_k], attn: [B, H, Lq, Lk]
        x, self.attention_weights = scaled_dot_product_attention(q, k, v, mask)
        
        # 4. Concatenate heads
        # [B, H, Lq, d_k] -> [B, Lq, H, d_k] -> [B, Lq, D]
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 5. Final linear layer
        # [B, Lq, D] -> [B, Lq, D]
        output = self.W_o(x)
        
        return output

# --- 3. Feed-Forward Network (FFN) Class ---
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.linear_2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        # [B, L, D_model] -> [B, L, D_ff]
        x = self.relu(self.linear_1(x))
        # [B, L, D_ff] -> [B, L, D_model]
        x = self.linear_2(x)
        return x

# Helper for Add&Norm (Residual Connection + Layer Norm)
class SublayerConnection(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, sublayer):
        # LayerNorm(x + sublayer(x))
        return x + sublayer(self.norm(x))

In [ ]:
# --- 1. The Encoder Layer ---
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        # Self-Attention
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model) for _ in range(2)])
        
    def forward(self, x, src_mask):
        # 1. Self-Attention Sublayer
        # Input is Q=x, K=x, V=x
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask=src_mask))
        
        # 2. Feed-Forward Sublayer
        x = self.sublayer[1](x, self.feed_forward)
        return x

# --- 2. The Decoder Layer ---
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super().__init__()
        # Masked Self-Attention
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # Cross-Attention
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model) for _ in range(3)])

    def forward(self, x, memory, src_mask, tgt_mask):
        # 1. Masked Self-Attention Sublayer
        # Input is Q=x, K=x, V=x
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask=tgt_mask))
        
        # 2. Cross-Attention Sublayer (Q=x, K=memory, V=memory)
        # 'memory' is the output of the Encoder
        x = self.sublayer[1](x, lambda x: self.cross_attn(x, memory, memory, mask=src_mask))
        
        # 3. Feed-Forward Sublayer
        x = self.sublayer[2](x, self.feed_forward)
        return x

In [ ]:
# Helper for Positional Encoding (Only for Decoder)
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        
        # --- Full PE Matrix Calculation ---
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-(np.log(10000.0) / d_model))
        )
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0) # Shape: [1, max_len, d_model]
        
        # Register the buffer
        self.register_buffer('pe', pe) 

    # CORRECT FORWARD METHOD: Only takes 'x' and adds the PE
    def forward(self, x):
        # x is [B, L, D_model]. Add the PE to the input embeddings.
        # [B, L, D_model] + [1, L, D_model]
        x = x + self.pe[:, :x.size(1), :]
        return x

# --- 1. Full Encoder Module ---
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        # **CRITICAL**: No Positional Encoding added here for content-addressability test.
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, src_mask):
        # [B, L] -> [B, L, D_model]
        x = self.embedding(src)
        
        for layer in self.layers:
            # Ensure src_mask is [B, 1, L_src] and ready for broadcast inside MHA
            x = layer(x, src_mask)
        
        return self.norm(x)

# --- 2. Full Decoder Module ---
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pe = PositionalEncoding(d_model, max_len=max_seq_len) # PE is typically required for generation
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, memory, src_mask, tgt_mask):
        # [B, L] -> [B, L, D_model]
        x = self.embedding(tgt)
        x = self.pe(x) # Add Positional Encoding
        
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
            
        return self.norm(x)

# --- 3. CrossAttentionLookupModel (The Final Model) ---
class CrossAttentionLookupModel(nn.Module):
    # FIX APPLIED HERE: Added all 6 required parameters to the __init__ signature
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len): 
        super().__init__()
        self.encoder = Encoder(vocab_size, d_model, num_heads, d_ff, num_layers)
        self.decoder = Decoder(vocab_size, d_model, num_heads, d_ff, num_layers, max_seq_len)
        self.projection = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt, src_padding_mask, tgt_padding_mask):
        # 1. Prepare 4D Encoder Padding Mask
        # FROM: [B, L_src] -> [B, 1, L_src] (Incorrect)
        # TO:   [B, L_src] -> [B, 1, 1, L_src] (Correct for MHA broadcast)
        # Convert boolean mask (True=PAD) to float mask (1=PAD) for MHA if needed, but for padded_fill, the boolean mask is fine.
        # We need the 4D shape: [B, 1, 1, L_src]
        encoder_src_mask = src_padding_mask.unsqueeze(1).unsqueeze(2) 
        
        # Encoder Output: [B, L_src, D_model]
        memory = self.encoder(src, encoder_src_mask) # Pass the 4D mask

        # 2. Prepare 4D Decoder Self-Attention Mask (Causal + Padding)
        tgt_seq_len = tgt.size(1)
        tgt_causal_mask = (torch.triu(torch.ones(tgt_seq_len, tgt_seq_len, device=src.device), diagonal=1) == 1) # [L_tgt, L_tgt]
        
        # Combine causal and padding masks: [B, 1, L_tgt, L_tgt]
        # full_tgt_mask = tgt_causal_mask.unsqueeze(0).unsqueeze(1) | tgt_padding_mask.unsqueeze(1).unsqueeze(2) # Original logic was overcomplicating the broadcast
        
        # A simple, robust way for Self-Attention Mask: [B, 1, L_tgt, L_tgt]
        # The padding mask must be expanded to [B, 1, L_tgt, 1] to mask the queries, 
        # and [B, 1, 1, L_tgt] to mask the keys/values.
        # Let's use the explicit 4D mask for the full self-attention weights.
        
        # Combine Causal Mask [L_tgt, L_tgt] and Padding Mask [B, L_tgt]
        # Causal mask: [1, 1, L_tgt, L_tgt]. Padding mask (for query tokens): [B, 1, L_tgt, 1].
        # Padding mask (for key tokens): [B, 1, 1, L_tgt].
        
        # Use simpler boolean mask logic for PyTorch MHA (which takes a 2D or 3D mask)
        # For simplicity and to match the 4D input of scaled_dot_product_attention:
        
        full_tgt_mask = tgt_causal_mask.unsqueeze(0).unsqueeze(1) # [1, 1, L_tgt, L_tgt]
        
        # Padding Mask for Keys/Values: [B, 1, 1, L_tgt]
        key_padding_mask = tgt_padding_mask.unsqueeze(1).unsqueeze(2)
        
        # Add key padding to causal mask:
        full_tgt_mask = full_tgt_mask | key_padding_mask # This results in [B, 1, L_tgt, L_tgt]

        # 3. Prepare 4D Cross-Attention Padding Mask (Same as encoder_src_mask, just renamed)
        final_src_mask_for_decoder = encoder_src_mask 

        decoder_output = self.decoder(
            tgt, 
            memory, 
            # Cross-Attention K/V padding: [B, 1, 1, L_src]
            final_src_mask_for_decoder,
            # Decoder Self-Attention mask: [B, 1, L_tgt, L_tgt]
            full_tgt_mask 
        )

        # Final linear projection
        output = self.projection(decoder_output)
        
        return output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. Initialization ---
model = CrossAttentionLookupModel(
    VOCAB_SIZE, D_MODEL, NUM_HEADS, D_FF, NUM_LAYERS, MAX_SEQ_LEN
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# Ignore [PAD] token in loss calculation
criterion = nn.CrossEntropyLoss(ignore_index=VOCAB['[PAD]'])

# Variables to track best performance
best_val_loss = float('inf')

In [ ]:
# --- 2. Training Loop ---
print("\n--- Starting Training ---")
for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    for enc_input, dec_input, dec_target, src_padding_mask, tgt_padding_mask in train_loader:
        enc_input, dec_input, dec_target = enc_input.to(device), dec_input.to(device), dec_target.to(device)
        src_padding_mask, tgt_padding_mask = src_padding_mask.to(device), tgt_padding_mask.to(device)

        optimizer.zero_grad()
        
        # Forward pass: output is [B, L_tgt, VOCAB_SIZE]
        output = model(enc_input, dec_input, src_padding_mask, tgt_padding_mask)
        
        # Reshape for CrossEntropyLoss: [B*L, VOCAB_SIZE] and [B*L]
        loss = criterion(output.reshape(-1, VOCAB_SIZE), dec_target.reshape(-1))
        
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        
    avg_train_loss = total_train_loss / len(train_loader)
    
    # --- Validation ---
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for enc_input, dec_input, dec_target, src_padding_mask, tgt_padding_mask in val_loader:
            enc_input, dec_input, dec_target = enc_input.to(device), dec_input.to(device), dec_target.to(device)
            src_padding_mask, tgt_padding_mask = src_padding_mask.to(device), tgt_padding_mask.to(device)

            output = model(enc_input, dec_input, src_padding_mask, tgt_padding_mask)
            loss = criterion(output.reshape(-1, VOCAB_SIZE), dec_target.reshape(-1))
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    
    # Track and save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_lookup_model.pth')

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

print("--- Training Complete ---")
# Load the best model weights
model.load_state_dict(torch.load('best_lookup_model.pth'))
model.eval()

In [ ]:
# --- 1. Corrected Inference Function (Snippet) ---
def greedy_decode(model, enc_input, max_len):
    model.eval()
    with torch.no_grad():
        src_padding_mask = (enc_input == VOCAB['[PAD]']).to(device)
        
        # 1. Encoder forward pass: Mask for Encoder Self-Attention [B, 1, 1, L_src]
        memory = model.encoder(enc_input, src_padding_mask.unsqueeze(1).unsqueeze(2)) 
        
        dec_input = torch.tensor([[VOCAB['[SOS]']]], dtype=torch.long, device=device)
        
        # 2. Cross-Attention Mask: Must match the Encoder Self-Attention shape
        # [B, 1, 1, L_src]
        final_src_mask_for_decoder = src_padding_mask.unsqueeze(1).unsqueeze(2) # Corrected!
        
        # ... (rest of the loop) ...
        for _ in range(max_len):
            tgt_len = dec_input.size(1)
            
            # 1. Causal mask definition (MISSING LINE)
            tgt_causal_mask = (torch.triu(torch.ones(tgt_len, tgt_len), diagonal=1) == 1).to(device)
            # ...
            decoder_output = model.decoder(
                dec_input, 
                memory, 
                # Use the correct 4D mask for K/V padding
                final_src_mask_for_decoder, 
                # Causal mask: [1, 1, L_tgt, L_tgt]
                tgt_causal_mask.unsqueeze(0).unsqueeze(1)
            )
            
            # Project to vocab size: [1, L_tgt, VOCAB_SIZE]
            output = model.projection(decoder_output[:, -1:]) # Only take the last token's output
            
            # Greedy search: get the token with highest probability
            next_token = torch.argmax(output, dim=-1) # [1, 1]
            
            # Append to sequence
            dec_input = torch.cat([dec_input, next_token], dim=1)
            
            # Stop condition
            if next_token.item() == VOCAB['[EOS]']:
                break
                
        # Return the generated sequence (excluding [SOS])
        return dec_input[0, 1:]

# --- 2. The Scrambling Test Execution ---
print("\n--- Scrambling Test (Content-Addressability) ---")
# Get one sample from the test set
test_sample = next(iter(test_loader))
enc_input_batch, _, dec_target_batch, _, _ = test_sample
original_enc_input = enc_input_batch[0:1].to(device) # [1, L_src]
true_target = [ID_TO_TOKEN[i.item()] for i in dec_target_batch[0] if i.item() != VOCAB['[PAD]']]

# Original Order Inference
original_output = greedy_decode(model, original_enc_input, max_len=MAX_SEQ_LEN)
original_output_tokens = [ID_TO_TOKEN[i.item()] for i in original_output if i.item() != VOCAB['[EOS]']]

# Scramble the Encoder Input (excluding the first token 'NAME' for a stable query)
# and the padding tokens.
enc_list = original_enc_input[0].cpu().numpy().tolist()
non_pad_tokens = [t for t in enc_list if t != VOCAB['[PAD]']]
first_token = non_pad_tokens[0]
scramblable_tokens = non_pad_tokens[1:]

np.random.shuffle(scramblable_tokens)
scrambled_list = [first_token] + scramblable_tokens
# Pad back to original length
scrambled_list.extend([VOCAB['[PAD]']] * (original_enc_input.size(1) - len(scrambled_list)))

scrambled_enc_input = torch.tensor([scrambled_list], dtype=torch.long, device=device)

# Scrambled Order Inference
scrambled_output = greedy_decode(model, scrambled_enc_input, max_len=MAX_SEQ_LEN)
scrambled_output_tokens = [ID_TO_TOKEN[i.item()] for i in scrambled_output if i.item() != VOCAB['[EOS]']]

print(f"True Target: {true_target}")
print("-" * 30)

# Print results
print(f"Original Input: {' '.join([ID_TO_TOKEN[i.item()] for i in original_enc_input[0] if i.item() != VOCAB['[PAD]']])}")
print(f"Original Output: {' '.join(original_output_tokens)}")
print("-" * 30)
print(f"Scrambled Input: {' '.join([ID_TO_TOKEN[i.item()] for i in scrambled_enc_input[0] if i.item() != VOCAB['[PAD]']])}")
print(f"Scrambled Output: {' '.join(scrambled_output_tokens)}")
print("-" * 30)
print(f"Outputs Identical: **{original_output_tokens == scrambled_output_tokens}** (Expected: True)")

In [ ]:
# --- 1. Attention Hook Setup ---
# The target layer is the cross_attn in the first (and only) DecoderLayer
cross_attn_module = model.decoder.layers[0].cross_attn

# A list to store the attention weights
attention_weights_storage = []

def capture_attention_hook(module, input, output):
    # output[1] is the attention weights tensor from MultiHeadAttention
    # Shape: [B, H, L_tgt, L_src]
    attention_weights_storage.append(module.attention_weights.detach().cpu())

# Register the forward hook
hook = cross_attn_module.register_forward_hook(capture_attention_hook)

# --- 2. Extraction Run ---
# Run a single batch from the test_loader
test_batch = next(iter(test_loader))
enc_input, dec_input, dec_target, src_padding_mask, tgt_padding_mask = [t.to(device) for t in test_batch]

# Run model in eval mode to trigger the hook
with torch.no_grad():
    _ = model(enc_input, dec_input, src_padding_mask, tgt_padding_mask)

# Remove the hook after extraction
hook.remove()

# Extract the attention matrix for the first head, first batch item
# [1, H, L_tgt, L_src] -> [L_tgt, L_src]
attention_matrix = attention_weights_storage[0][0, 0].squeeze().numpy()

# Get token labels for the plot
# First sample in the batch
enc_tokens_ids = enc_input[0].cpu().numpy().tolist()
dec_tokens_ids = dec_input[0].cpu().numpy().tolist()

# Filter out PAD tokens and map to actual tokens
enc_tokens = [ID_TO_TOKEN[i] for i in enc_tokens_ids if i != VOCAB['[PAD]']]
# dec_input has [SOS, Target_Val, EOS, PAD...]
dec_tokens = [ID_TO_TOKEN[i] for i in dec_tokens_ids if i != VOCAB['[PAD]']]

# Trim attention matrix to non-padded tokens
L_tgt = len(dec_tokens)
L_src = len(enc_tokens)
attention_matrix = attention_matrix[:L_tgt, :L_src]


# --- 3. Visualization ---
def plot_cross_attention(attention_matrix, enc_tokens, dec_tokens):
    """Plots the cross-attention matrix as a heatmap."""
    plt.figure(figsize=(8, 6))
    
    # Use Seaborn for a nicer heatmap
    sns.heatmap(
        attention_matrix, 
        xticklabels=enc_tokens, 
        yticklabels=dec_tokens, 
        cmap="Blues", 
        linewidths=.5, 
        linecolor='black',
        annot=True,
        fmt=".2f",
        cbar_kws={'label': 'Attention Weight'}
    )
    
    plt.xlabel('Encoder Input Tokens (K/V)')
    plt.ylabel('Decoder Input Tokens (Q)')
    plt.title('Cross-Attention Heatmap (Query-to-Fact)')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Execute plot_cross_attention to display the key heatmap.
plot_cross_attention(attention_matrix, enc_tokens, dec_tokens)
#